In this app we:
* Edit a [Planar Straight Line Graph](https://en.wikipedia.org/wiki/Planar_straight-line_graph) with the [erdc/ipymesh](https://github.com/erdc/ipymesh) widget
* Generate a triangular mesh with the [triangle](https://www.cs.cmu.edu/~quake/triangle.html) mesh generator using the [triangle python bindings](https://rufat.be/triangle/index.html)

Find the code [here](https://github.com/pbugnion/voila-gallery/blob/master/mesh-generation/index.ipynb).

In [1]:
from ipymesh import PSLGEditor
editor = PSLGEditor(vertices=[(0.0,0.0),
                              (0.0,1.0),
                              (1.0,1.0),
                              (1.0,0.0)],
                    vertexFlags=[1,1,2,2],
                    segments=[(0,1),
                              (1,2),
                              (2,3),
                              (3,0)],
                    segmentFlags=[1,1,2,1],
                    regions=[(0.5,0.5)],
                    regionFlags=[1],
                    regionTypes=[1,2,0],
                    boundaryTypes=[1,2,0],
                    Lx=1.05,
                    Ly=1.05,
                    x0=-0.15,
                    y0=-0.15,
                    width=500,
                    height=500)

In [2]:
import ipywidgets
import triangle as tr
import matplotlib.pyplot as plt
from bqplot import (
    LinearScale, OrdinalColorScale, Lines, Axis, ColorAxis, Figure
)
fig=Figure()
def genMesh(b=None):
    global editor,fig
    pslg = dict(vertices=editor.vertices,
                vertex_attributes=editor.vertexFlags,
                segments=editor.segments,
                segment_markers=editor.segmentFlags)
    if editor.holes:
        pslg['holes'] = editor.holes
    if editor.regions:
        pslg['regions'] = [[r[0],r[1],rF, 0.1] for (r,rF) in 
                           zip(editor.regions,editor.regionFlags)]
    mesh = tr.triangulate(pslg,'pq')
    size = 20
    x_data = [[x1,x2] for x1,x2 in zip(mesh['vertices'][mesh['segments'][:,0],0],
                                       mesh['vertices'][mesh['segments'][:,1],0])]
    y_data = [[y1,y2] for y1,y2 in zip(mesh['vertices'][mesh['segments'][:,0],1],
                                       mesh['vertices'][mesh['segments'][:,1],1])]
    colorsList = [ci[0] for ci in mesh['segment_markers']]
    edges=set()
    for t in mesh['triangles']:
        for n in range(3):
            e = [t[n],t[(n+1)%3]]
            e.sort()
            edges.add(tuple(e))
    for e in edges:
        x_data.append([mesh['vertices'][e[0],0],mesh['vertices'][e[1],0]])
        y_data.append([mesh['vertices'][e[0],1],mesh['vertices'][e[1],1]])
        colorsList.append(0)
    assert(len(colorsList) == len(x_data))
    x_sc = LinearScale()
    y_sc = LinearScale()
    c_sc = OrdinalColorScale(domain=[0,1,2])
    line = Lines(x=x_data, 
                 y=y_data, 
                 color=colorsList,
                 scales={'x': x_sc, 
                         'y': y_sc, 
                         'color': c_sc},
                 stroke_width=3,  
                 display_legend=False)
    ax_x = Axis(scale=x_sc, grid_lines='solid', label='x')
    ax_y = Axis(scale=y_sc, orientation='vertical', tick_format='0.2f',
                grid_lines='solid', label='y')
    ax_c = ColorAxis(scale=c_sc, side='right', visible=True)

    fig.marks=[line]
    fig.axes=[ax_x, ax_y, ax_c]
    fig.max_aspect_ratio = 1
    fig.min_aspect_ratio = 1
genMesh()
button  = ipywidgets.Button(
    description='Generate Mesh',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Regenerate the mesh'
)
button.on_click(genMesh)

In [3]:
from ipywidgets import Tab,VBox
app=Tab(children=[VBox(children=[button, editor]),
                  fig])
app.set_title(0,"Graph Editor")
app.set_title(1,"Mesh")
display(app)